# Political Labelling

This Script categorices Twitter User IDs with a political affiliation. For that, We use a list of political influencers previously categorized as left, center, or right by La Silla Vacia, a Colombian news outlet and and ideological classification of congressmen based on the "Partidos Políticos en Colombia" database fro the Documento CEDE #36 "Una caracterización histórica de los partidos políticos de Colombia: 1958-2022"
https://datoscede.uniandes.edu.co/es/catalogo-de-microdata


In [1]:
import pandas as pd
import pickle

In [2]:
partidos = pd.read_excel("~/clasificacion_partidos_v1.xlsx","Sheet1")
partidos = partidos.loc[:,['codigo_partido','ideologia']]

congresistas = pd.read_excel("~/Twitter Congresistas.xlsx","Sheet1")
congresistas = congresistas.loc[:,('Partido', 'Twitter')]

# We load the tweets_lite DataFrame for the analysis
retweets = pd.read_pickle('/mnt/disk2/Data/Tweets_DataFrames/retweets_lite.gzip', compression='gzip')

# We load the map that relates an ID to a political Label
with open("/mnt/disk2/Data/Pickle/User_Dicts/mapa.pkl", "rb") as file:
    mapa = pickle.load(file)

In [3]:
def partidos_dict(x):
    
    d = {
        "PartidoSocialdeUnidadNacional":20050002,#PartidodelaU
        "CentroDemocrático":20130001,#CentroDemocrático
        "ListadelaDecencia":20180008,#ListadelaDecencia
        "AlianzaVerde":20090002,#AlianzaVerde
        "PartidoConservadorColombiano":18490002,#PartidoConservadorColombiano
        "PartidoLiberalColombiano":18480001,#PartidoLiberalColombiano
        "ColombiaJustaLibres":20170001,#ColombiaJustaLibres
        "PoloDemocráticoAlternativo":20050001,#PoloDemocráticoAlternativo
        "PartidoCambioRadical":20030001,#PoloCambioRadical
        "ConsejoComunitariodeComunidadesNegrasPlayaRenaciente":20180014,#ConsejoComunitarioPlayaRenaciente
        "Comunes":20170003,#PartidoComunes
        "MovimientoAlternativoIndígenaySocial":20130002,#MAIS
        "ConsejoComunitarioLaMamuncia":20180029,#ConsejoComunitariolaMamuncia(wrongnameinexternaldatabase)
        "CoaliciónAlternativaSantandereana":20180042,#CoaliciónAlternativaSantandereana
        "MovimientoIndependientedeRenovaciónAbsoluta":20000036,#MIRA
        "OpciónCiudadana":20090001#Partido Opción Ciudadana
    }
    
    try:
        return d[x]
    except KeyError:
        return 0

def idelogias_dict(x):
    d = {
        1:'Izquierda',
        2:'Derecha',
        3:'Centro',
        4:'Sin Clasificar'
    }
    
    try:
        return d[x]
    except KeyError:
        return 'nada'

In [4]:
# Diccionario con ID Nombre de twittero
user_name = (
    retweets[['Referenced Tweet Author ID', 'Referenced Tweet Author Name']]
    .drop_duplicates()
    .astype({'Referenced Tweet Author ID':int})
    .set_index('Referenced Tweet Author ID')
    .to_dict()['Referenced Tweet Author Name']
)

# Limpiar nombres para agregar el ID de partido
congresistas.Partido = congresistas.Partido.str.replace(' ','')
congresistas['Partido ID'] = congresistas['Partido'].apply(partidos_dict)
congresistas_new = congresistas.merge(partidos,left_on='Partido ID', right_on='codigo_partido', how = 'left')

# Obetenemos la ideología
congresistas_new['Afiliacion'] = congresistas_new.ideologia.apply(idelogias_dict)
mapa_2 = (
    congresistas_new.merge(retweets, left_on='Twitter', right_on='Referenced Tweet Author Name')
    .loc[:,('Referenced Tweet Author ID','Afiliacion')]
    .query("Afiliacion != 'nada'")
    .drop_duplicates()
    .astype({'Referenced Tweet Author ID':int})
    .set_index('Referenced Tweet Author ID')
    .to_dict()['Afiliacion']
)

In [5]:
common_keys = list(set(mapa.keys()).intersection(set(mapa_2.keys())))
print(f"Hay {len(mapa)} usuarios identificados por la Silla")
print(f"Hay {len(mapa_2)} congresistas identificados según su partido")
print(f"Hay {len(common_keys)} usuarios identificados por ambas fuentes")
print('')
for key in common_keys:
    if mapa[key] != mapa_2[key]:
        # Usar el criterio de la silla vacía
        print(f"{user_name[key]}: Silla dice {mapa[key]} pero partido es {mapa_2[key]}")
    mapa_2.pop(key)

# Unir ambos diccioanrios
mapa_full = mapa_2 | mapa
print('')
print(f"Total de dicionarios {len(mapa_full)}")

# Guardar
with open("/mnt/disk2/Data/Pickle/User_Dicts/mapa_full.pkl", "wb") as file:
    pickle.dump(mapa_full, file)

Hay 93 usuarios identificados por la Silla
Hay 228 congresistas identificados según su partido
Hay 15 usuarios identificados por ambas fuentes

JERobledo: Silla dice Centro pero partido es Izquierda
RoyBarreras: Silla dice Izquierda pero partido es Derecha
angelamrobledo: Silla dice Centro pero partido es Izquierda
AABenedetti: Silla dice Izquierda pero partido es Derecha
intiasprilla: Silla dice Izquierda pero partido es Centro

Total de dicionarios 306
